In [1]:
import pandas as pd
from sklearn import ensemble, preprocessing, linear_model
import random
import glob

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:

df = pd.concat([pd.read_csv(i) for i in glob.glob('/media/td/Samsung_T5/sports/nba/nn_architectures*')])


In [3]:
df.sort_values('accuracy', ascending = False).head(20)


,convolutional_kernel_size,convolutional_pool_size,pooling_algorithm,convolution_type,recurrent_type,use_x1,use_x2,optimizer_algorithm,conv_layers_per_pooling_layer,batchnorm,block_1_activations,block_2_activations,merge_activations,top_block_activations,block_1_dropout,block_2_dropout,merge_block_dropout,top_block_dropout,merge_block_width,block_2_width,top_block_width,recurrent_layers_width,block_1_dense_layers_width,merge_block_layers,block_2_layers,convolutional_filters,dense_top_layers,convolutional_layers,recurrent_layers,block_1_dense_layers,block_1_dropout_amount,block_2_dropout_amount,merge_block_dropout_amount,top_block_dropout_amount,input_shape1,input_shape2,history_length,timeseries_scaled,general_feature_encoding_size,rotate_history_data,num_of_params,model_training_time,accuracy
8,9,2,layers.MaxPooling1D,layers.LocallyConnected1D,layers.SimpleRNN,0,1,optimizers.Nadam(),0,0,tanh,sigmoid,softplus,softplus,layers.GaussianDropout,layers.AlphaDropout,layers.GaussianDropout,layers.AlphaDropout,42,221,124,190,71,1,1,23,0,1,1,1,0.017621,0.317456,0.796645,0.374111,"(81, 16)","(2500,)",16.0,False,NaN,1.0,552943.0,2.261715,0.662142
1,5,4,NaN,layers.LocallyConnected1D,layers.GRU,1,1,optimizers.Adagrad(),1,1,tanh,selu,tanh,relu,NaN,layers.Dropout,layers.Dropout,NaN,23,152,145,112,117,1,0,127,0,0,2,0,0.000362,0.616564,0.329746,0.367660,"(81, 16)","(2500,)",16.0,False,NaN,1.0,709547.0,36.471948,0.656109
10,9,1,NaN,layers.SeparableConv1D,layers.GRU,0,1,optimizers.RMSprop(),1,0,tanh,hard_sigmoid,sigmoid,sigmoid,layers.GaussianNoise,layers.GaussianDropout,layers.AlphaDropout,layers.AlphaDropout,134,65,106,49,231,0,0,23,0,2,2,1,0.526513,0.474967,0.554669,0.792413,"(81, 8)","(128,)",8.0,True,128.0,1.0,8451.0,1.260044,0.650075
9,8,2,NaN,layers.Conv1D,layers.LSTM,0,1,optimizers.Adagrad(),0,0,softsign,relu,hard_sigmoid,softplus,layers.Dropout,layers.GaussianNoise,layers.GaussianNoise,layers.AlphaDropout,87,238,155,218,156,0,2,97,2,1,1,1,0.668271,0.769491,0.231755,0.702947,"(81, 32)","(16,)",32.0,True,16.0,1.0,122309.0,2.841258,0.638009
5,1,1,layers.MaxPooling1D,layers.Conv1D,layers.GRU,0,1,optimizers.Adadelta(),1,0,elu,elu,relu,sigmoid,layers.AlphaDropout,layers.GaussianDropout,layers.GaussianNoise,layers.Dropout,193,120,112,124,232,0,1,91,1,2,2,2,0.309005,0.011788,0.271978,0.522028,"(4, 81)","(32,)",4.0,False,32.0,0.0,17625.0,0.952629,0.633484
37,5,3,layers.MaxPooling1D,layers.LocallyConnected1D,layers.LSTM,0,1,optimizers.Adadelta(),2,0,relu,hard_sigmoid,selu,hard_sigmoid,layers.Dropout,layers.GaussianDropout,layers.GaussianNoise,layers.AlphaDropout,213,44,42,224,245,0,1,114,1,0,0,1,0.377385,0.216143,0.389044,0.548042,"(32, 81)","(4,)",32.0,False,4.0,0.0,2153.0,2.998746,0.633484
11,3,2,layers.MaxPooling1D,layers.Conv1D,layers.GRU,0,1,optimizers.RMSprop(),0,0,sigmoid,elu,softplus,elu,NaN,layers.Dropout,layers.GaussianNoise,layers.AlphaDropout,19,234,98,34,118,0,1,70,2,0,1,0,0.374066,0.630407,0.128391,0.353482,"(16, 81)","(2500,)",16.0,True,NaN,0.0,618065.0,2.689170,0.630468
29,8,3,layers.MaxPooling1D,layers.SeparableConv1D,layers.LSTM,0,1,optimizers.Adamax(),0,0,softplus,tanh,softplus,selu,layers.GaussianNoise,layers.AlphaDropout,layers.GaussianDropout,layers.GaussianDropout,112,184,182,74,135,1,0,112,1,1,0,1,0.023354,0.357762,0.045229,0.097861,"(81, 16)","(128,)",16.0,True,128.0,1.0,57589.0,2.295421,0.624434
6,2,4,layers.MaxPooling1D,layers.LocallyConnected1D,layers.GRU,1,0,optimizers.Adamax(),1,0,exponential,hard_sigmoid,softplus,hard_sigmoid,layers.AlphaDropout,NaN,layers.Dropout,layers.Dropout,76,124,215,163,133,1,2,111,2,0,1,0,0.279095,0.487088,0.122549,0.650941,"(81, 16)","(64,)",16.0,False,64.0,1.0,1154735.0,23.806959,0.615385
35,2,4,layers.MaxPooling1D,layers.SeparableConv1D,layers.GRU,1,1,optimizers.RMSprop(),1,1,tanh,softplus,hard_sigmoid,softsign,layers.GaussianNoise,NaN,layers.GaussianDropout,layers.GaussianNoise,206,139,36,137,66,0,1,92,1,2,0,2,0.502495,0.569988,0.506366,0.679372,"(81, 32)","(32,)",32.0,True,32.0,1.0,1944

In [4]:

df = df.fillna(-1)
options = dict()
encoders = dict()

for i in df.columns:
    if df[i].dtype == 'object':
        encoder = preprocessing.LabelEncoder()
        df[i] = encoder.fit_transform(df[i].astype(str))
        encoders[i] = encoder
    options[i] = list(set(df[i]))
        

In [5]:
df.head()

,convolutional_kernel_size,convolutional_pool_size,pooling_algorithm,convolution_type,recurrent_type,use_x1,use_x2,optimizer_algorithm,conv_layers_per_pooling_layer,batchnorm,block_1_activations,block_2_activations,merge_activations,top_block_activations,block_1_dropout,block_2_dropout,merge_block_dropout,top_block_dropout,merge_block_width,block_2_width,top_block_width,recurrent_layers_width,block_1_dense_layers_width,merge_block_layers,block_2_layers,convolutional_filters,dense_top_layers,convolutional_layers,recurrent_layers,block_1_dense_layers,block_1_dropout_amount,block_2_dropout_amount,merge_block_dropout_amount,top_block_dropout_amount,input_shape1,input_shape2,history_length,timeseries_scaled,general_feature_encoding_size,rotate_history_data,num_of_params,model_training_time,accuracy
0,8,2,0,2,0,1,1,0,0,1,7,5,7,2,1,1,1,1,176,207,102,194,59,0,1,54,2,2,2,1,0.773785,0.168571,0.236409,0.024964,6,6,4.0,2,8.0,1.0,2039997.0,56.848459,0.601810
1,5,4,0,1,0,1,1,1,1,1,8,4,7,3,0,2,2,0,23,152,145,112,117,1,0,127,0,0,2,0,0.000362,0.616564,0.329746,0.367660,4,2,16.0,1,-1.0,1.0,709547.0,36.471948,0.656109
2,3,4,0,0,2,1,0,5,1,1,7,1,6,4,3,0,0,3,223,255,218,19,125,0,1,28,0,2,2,0,0.607093,0.370030,0.362845,0.694677,8,2,8.0,1,-1.0,1.0,3613.0,6.459976,0.556561
3,5,4,1,2,0,0,1,5,1,0,3,0,6,7,3,4,2,1,119,45,22,159,108,1,2,85,2,0,0,1,0.231992,0.763073,0.004458,0.012149,4,3,16.0,2,32.0,1.0,5096.0,2.691719,0.603318
4,1,4,0,2,0,0,1,5,2,1,6,7,3,4,1,4,3,4,188,221,43,235,225,1,1,56,0,0,0,2,0.311157,0.347078,0.124723,0.149725,7,6,64.0,1,8.0,1.0,3095.0,1.602416,0.586727


In [6]:

# rf = ensemble.RandomForestRegressor()
model = linear_model.ElasticNet()
x = df.drop('accuracy', axis = 1)

x2 = x.copy()
for i1 in x.columns:
    for i2 in x.columns:
        x2[f'prod_of_{i1}_{i2}'] = x2[i1] * x2[i2]
y = df['accuracy']

model.fit(x, y)


ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [7]:

recs = []
for n in range(1000):
    rec = []
    for i in x.columns:
        rec.append(random.choice(options[i]))
    recs.append(rec)
    

In [9]:

df_rec = pd.DataFrame(data = recs,
                     columns = x.columns)
df_rec.head()

,convolutional_kernel_size,convolutional_pool_size,pooling_algorithm,convolution_type,recurrent_type,use_x1,use_x2,optimizer_algorithm,conv_layers_per_pooling_layer,batchnorm,block_1_activations,block_2_activations,merge_activations,top_block_activations,block_1_dropout,block_2_dropout,merge_block_dropout,top_block_dropout,merge_block_width,block_2_width,top_block_width,recurrent_layers_width,block_1_dense_layers_width,merge_block_layers,block_2_layers,convolutional_filters,dense_top_layers,convolutional_layers,recurrent_layers,block_1_dense_layers,block_1_dropout_amount,block_2_dropout_amount,merge_block_dropout_amount,top_block_dropout_amount,input_shape1,input_shape2,history_length,timeseries_scaled,general_feature_encoding_size,rotate_history_data,num_of_params,model_training_time
0,8,3,layers.MaxPooling1D,layers.SeparableConv1D,layers.SimpleRNN,0,1,optimizers.Nadam(),1,1,softplus,softsign,sigmoid,selu,-1,-1,layers.AlphaDropout,layers.GaussianNoise,164,203,146,19,49,1,2,23,1,1,0,2,0.773785,0.569988,0.018958,0.018909,"(81, 8)","(16,)",4.0,-1,16.0,-1.0,209441.0,2.691719
1,1,3,-1,layers.SeparableConv1D,layers.SimpleRNN,1,0,optimizers.Adagrad(),0,0,tanh,sigmoid,tanh,selu,layers.AlphaDropout,layers.Dropout,layers.GaussianDropout,-1,65,166,217,144,82,0,2,66,0,0,1,1,0.183318,0.607368,0.323049,0.679372,"(81, 64)","(16,)",16.0,False,32.0,1.0,139065.0,2.689170
2,5,3,layers.MaxPooling1D,layers.LocallyConnected1D,layers.GRU,1,1,optimizers.Adamax(),2,0,elu,softsign,elu,hard_sigmoid,layers.AlphaDropout,layers.GaussianDropout,layers.AlphaDropout,layers.Dropout,62,52,53,145,229,0,2,27,2,0,1,1,0.377385,0.487088,0.587144,0.760042,"(81, 32)","(64,)",64.0,True,-1.0,1.0,393187.0,2.261715
3,3,4,layers.MaxPooling1D,layers.SeparableConv1D,layers.GRU,1,1,optimizers.Adamax(),2,0,elu,hard_sigmoid,hard_sigmoid,tanh,layers.Dropout,-1,layers.GaussianNoise,-1,16,238,152,163,59,1,1,84,1,2,2,0,0.183318,0.449211,0.576362,0.271564,"(81, 8)","(16,)",-1.0,True,4.0,-1.0,1057945.0,6.383539
4,8,1,layers.MaxPooling1D,layers.Conv1D,layers.SimpleRNN,0,1,optimizers.RMSprop(),1,0,sigmoid,sigmoid,sigmoid,selu,layers.Dropout,layers.AlphaDropout,layers.Dropout,layers.GaussianDropout,193,207,164,124,16,1,2,124,0,2,0,0,0.759307,0.369651,0.061024,0.497883,"(4, 81)","(2500,)",4.0,False,32.0,-1.0,3727806.0,19.225342


In [8]:


df_rec_copy = df_rec.copy()

for i in encoders.keys():
    df_rec_copy[i] = encoders[i].transform(df_rec_copy[i])

df_rec['accuracy'] = model.predict(df_rec_copy)

df_rec.sort_values('accuracy', ascending = False).head(20)


ValueError: y contains previously unseen labels: -1